In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Clean data

In [10]:
loaded_df = pd.read_csv('leiden_publications.csv')
loaded_df["id"] = range(len(loaded_df))
loaded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124064 entries, 0 to 124063
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   identifier   124064 non-null  object
 1   datestamp    124064 non-null  object
 2   creator      122941 non-null  object
 3   title        123682 non-null  object
 4   language     118846 non-null  object
 5   date         123684 non-null  object
 6   type         123681 non-null  object
 7   subject      46996 non-null   object
 8   source       99936 non-null   object
 9   description  116044 non-null  object
 10  license      81863 non-null   object
 11  format       85082 non-null   object
 12  contributor  15350 non-null   object
 13  publisher    8362 non-null    object
 14  alternative  132 non-null     object
 15  relation     1620 non-null    object
 16  id           124064 non-null  int64 
dtypes: int64(1), object(16)
memory usage: 16.1+ MB


In [11]:
# TODO:
# - [x] Identifier column
# - [x] datestamp column 
# - [x] source column
# - [x] language column
# - [x] title column
# - [x] description column
# - [ ] type column

df = loaded_df.copy()

# parse the identifier colum into new columns: ['oai:scholarlypublications.universiteitleiden.nl:item_4172863', 'doi:10.3390/ijerph21121561', 'lucris-id:\\xa01338718282', 'https://hdl.handle.net/1887/4172863']
# - oai
df['oai'] = df['identifier'].str.extract(r'(oai:\S+)')
# - doi
df['doi'] = df['identifier'].str.extract(r'(doi:\S+)')
# - lucris-id
df['lucris-id'] = df['identifier'].str.extract(r'(lucris-id:\S+)')
# - url
df['url'] = df['identifier'].str.extract(r'(https:\S+)')
# - remove the identifier column
df = df.drop(columns=['identifier'])

# Date column is in the form of: ['2024', '2026-02-11']
# - parse the date column into a new column. Get the second date (2026-02-11) if it exists, otherwise get the first date (2024)
# - convert the date column to a datetime object
df['date'] = df['date'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Type column is in the form of: ['Article in monograph or in proceedings', 'info:eu-repo/semantics/article', 'Text']
# - parse the type column into a new column with the info:eu-repo/semantics/article value. Extract only the article part

df['type'] = df['type'].str.extract(r'(info:eu-repo/semantics/\S+)')
# - remove the info:eu-repo/semantics/ part
df['type'] = df['type'].str.replace("info:eu-repo/semantics/", "").str.replace("',", "")
# - remove the type column

df = df[['oai', 'doi', 'lucris-id', 'url', 'date', 'source', 'language', 'title', 'description', "type", "id"]]

# remove ", ', , and \n from the description column
df['description'] = df['description'].str.replace("'", '').str.replace('"', '').str.replace(',', '').str.replace('\n', '')
df["title"] = df["title"].str.replace("'", '').str.replace('"', '').str.replace(',', '').str.replace('\n', '')

# # save cleaned data to a new parquet file
df.to_parquet('./data/leiden_publications_cleaned.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124064 entries, 0 to 124063
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   oai          124064 non-null  object        
 1   doi          71129 non-null   object        
 2   lucris-id    59686 non-null   object        
 3   url          123684 non-null  object        
 4   date         87613 non-null   datetime64[ns]
 5   source       99936 non-null   object        
 6   language     118846 non-null  object        
 7   title        123682 non-null  object        
 8   description  116044 non-null  object        
 9   type         123681 non-null  object        
 10  id           124064 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 10.4+ MB


In [12]:
df = loaded_df.copy()

import ast
def safe_parse_authors(text):
    if not isinstance(text, str):
        return []
    try:
        # If it's already in list format
        if text.startswith('[') and text.endswith(']'):
            return ast.literal_eval(text)
        # If it's a single author without brackets
        else:
            return [text]
    except:
        # If parsing fails, return the original as a single-item list
        return [text]
    
# give a id to every publication


df["creators_parsed"] = df["creator"].apply(safe_parse_authors)
df["subjects_parsed"] = df["subject"].apply(safe_parse_authors)

# now create a seperate dataframe with the authors and the ids of the publications, with every author on a seperate row


In [13]:
authors_df = df.explode('creators_parsed')[['creators_parsed', 'id']]
authors_df = authors_df.rename(columns={'creators_parsed': 'author'})

subjects_df = df.explode('subjects_parsed')[['subjects_parsed', 'id']]
subjects_df = subjects_df.rename(columns={'subjects_parsed': 'subject'})

# replace " with empty string
authors_df['author'] = authors_df['author'].str.replace('"', '')
subjects_df['subject'] = subjects_df['subject'].str.replace('"', '')

# save to csv file
authors_df.to_csv('./data/authors.csv', index=False)
subjects_df.to_csv('./data/subjects.csv', index=False)

In [14]:
import duckdb

# delete the database file if it exists
import os
if os.path.exists('data.db'):
    # ask the user if they want to delete the file
    delete = input("The database file already exists. Do you want to delete it? (y/n)")
    if delete == 'y':
        os.remove('data.db')
    else:
        print("Exiting the program.")
        raise KeyboardInterrupt
    

con = duckdb.connect("data.db")

# load the data into the database
con.execute("CREATE TABLE authors (author VARCHAR, id INTEGER)")
con.execute("CREATE TABLE subjects (subject VARCHAR, id INTEGER)")
#  0   oai          124064 non-null  object        
#  1   doi          71129 non-null   object        
#  2   lucris-id    59686 non-null   object        
#  3   url          123684 non-null  object        
#  4   date         87613 non-null   datetime64[ns]
#  5   source       99936 non-null   object        
#  6   language     118846 non-null  object        
#  7   title        123682 non-null  object        
#  8   description  116044 non-null  object        
#  9   type         123681 non-null  object  
# 10   id           124064 non-null  int64
con.execute("CREATE TABLE publications (oai VARCHAR, doi VARCHAR, lucris_id VARCHAR, url VARCHAR, date DATE, source VARCHAR, language VARCHAR, title VARCHAR, description VARCHAR, type VARCHAR, id INTEGER)")

con.execute("COPY authors FROM 'data/authors.csv' (HEADER)")
con.execute("COPY subjects FROM 'data/subjects.csv' (HEADER)")
con.execute("COPY publications FROM 'data/leiden_publications_cleaned.parquet'")


# join authors and subject on id
# query = """
# SELECT authors.author, subjects.subject
# FROM authors
# JOIN subjects
# ON authors.id = subjects.id
# WHERE authors.author = 'Lange, J. de'
# """

# result = con.execute(query)
# df = result.fetchdf()

# select the authors and the publication titles
query = """
SELECT authors.author, publications.title, publications.date, publications.type
FROM authors
JOIN publications
ON authors.id = publications.id
WHERE authors.author = 'Lange, J. de'
"""

result = con.execute(query)
df = result.fetchdf()

con.close()
df.head()

,author,title,date,type
0,"Lange, J. de",Literatuur,NaT,article
1,"Lange, J. de",De Bijlmermeer: Een voorbeeld van een besluitv...,NaT,article
2,"Lange, J. de",RANKL inhibition for giant cell lesions of the...,2022-11-01,article
3,"Lange, J. de",Oestrogen receptor expression distinguishes no...,2022-05-25,article
4,"Lange, J. de",ELOF1 is a transcription-coupled DNA repair fa...,2021-06-01,article
